<a href="https://colab.research.google.com/github/Avina20/Hematoma/blob/master/Hematoma_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
!pip install pillow==5.3.0
!pip install scipy==1.2.1
from scipy.misc import imread, imresize, imsave
import nibabel as nib
import matplotlib.pyplot as plt

In [ ]:
def window_ct (ct_scan, w_level=40, w_width=120):
    w_min = w_level - w_width / 2
    w_max = w_level + w_width / 2
    num_slices=ct_scan.shape[2]
    for s in range(num_slices):
        slice_s = ct_scan[:,:,s]
        slice_s = (slice_s - w_min)*(255/(w_max-w_min)) #or slice_s = (slice_s - (w_level-(w_width/2)))*(255/(w_width))
        slice_s[slice_s < 0]=0
        slice_s[slice_s > 255] = 255
        #slice_s=np.rot90(slice_s)
        ct_scan[:,:,s] = slice_s

    return ct_scan

In [ ]:
numSubj = 82
new_size = (512, 512)
window_specs=[40,120]

In [ ]:
currentDir = Path(os.getcwd())
datasetDir = str(Path(currentDir))

In [ ]:
# Reading labels
hemorrhage_diagnosis_df = pd.read_csv(
    Path(currentDir, 'hemorrhage_diagnosis_raw_ct.csv'))
hemorrhage_diagnosis_array = hemorrhage_diagnosis_df._get_values

In [ ]:
# reading images
train_path = Path('data')
image_path = train_path / 'image'
label_path = train_path / 'label'
if not train_path.exists():
    train_path.mkdir()
    image_path.mkdir()
    label_path.mkdir()

In [ ]:
import PIL
print(PIL.PILLOW_VERSION)

5.3.0


In [ ]:
counterI = 0
for sNo in range(0+49, numSubj+49):
    if sNo>58 and sNo<66: #no raw data were available for these subjects
        next
    else:
        #Loading the CT scan
        ct_dir_subj = Path(datasetDir,'ct_scans', "{0:0=3d}.nii".format(sNo))
        ct_scan_nifti = nib.load(str(ct_dir_subj))
        ct_scan = ct_scan_nifti.get_data()
        ct_scan = window_ct(ct_scan, window_specs[0], window_specs[1])

        #Loading the masks
        #masks_dir_subj = Path(datasetDir,'masks', "{0:0=3d}.nii".format(sNo))
        #masks_nifti = nib.load(str(masks_dir_subj))
        #masks = masks_nifti.get_data()
        idx = hemorrhage_diagnosis_array[:, 0] == sNo
        sliceNos = hemorrhage_diagnosis_array[idx, 1]
        NoHemorrhage = hemorrhage_diagnosis_array[idx, 7]
        #if sliceNos.size!=ct_scan.shape[2]:
            #print('Warning: the number of annotated slices does not equal the number of slices in NIFTI file!')

        for sliceI in range(0, sliceNos.size):
            # Saving the a given CT slice
            x = imresize(ct_scan[:,:,sliceI], new_size)
            imsave(image_path / (str(counterI) + '.png'), x)

            # Saving the segmentation for a given slice
            #segment_path = Path(masks_dir_subj,str(sliceNos[sliceI]) + '_HGE_Seg.jpg')
            #x = imresize(masks[:,:,sliceI], new_size)
            #imsave(label_path / (str(counterI) + '.png'), x)
            counterI = counterI+1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


In [ ]:
from google.colab import files

In [ ]:
!zip -r /content/img.zip /content/data/image
files.download("/content/img.zip")

updating: content/data/image/ (stored 0%)
updating: content/data/image/832.png (deflated 2%)
updating: content/data/image/270.png (deflated 7%)
updating: content/data/image/1926.png (deflated 7%)
updating: content/data/image/2456.png (deflated 1%)
updating: content/data/image/2638.png (deflated 0%)
updating: content/data/image/647.png (deflated 1%)
updating: content/data/image/787.png (deflated 0%)
updating: content/data/image/1798.png (deflated 0%)
updating: content/data/image/2528.png (deflated 1%)
updating: content/data/image/2262.png (deflated 1%)
updating: content/data/image/2634.png (deflated 2%)
updating: content/data/image/1965.png (deflated 16%)
updating: content/data/image/1044.png (deflated 1%)
updating: content/data/image/2369.png (deflated 1%)
updating: content/data/image/2429.png (deflated 0%)
updating: content/data/image/756.png (deflated 0%)
updating: content/data/image/139.png (deflated 1%)
updating: content/data/image/1529.png (deflated 21%)
updating: content/data/ima

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 60110, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [ ]:
!zip -r /content/label.zip /content/data/label
files.download("/content/label.zip")

  adding: content/data/label/ (stored 0%)
  adding: content/data/label/832.png (deflated 76%)
  adding: content/data/label/270.png (deflated 76%)
  adding: content/data/label/1926.png (deflated 76%)
  adding: content/data/label/2456.png (deflated 76%)
  adding: content/data/label/2638.png (deflated 76%)
  adding: content/data/label/647.png (deflated 76%)
  adding: content/data/label/787.png (deflated 22%)
  adding: content/data/label/1798.png (deflated 76%)
  adding: content/data/label/2528.png (deflated 76%)
  adding: content/data/label/2262.png (deflated 76%)
  adding: content/data/label/2634.png (deflated 76%)
  adding: content/data/label/1965.png (deflated 76%)
  adding: content/data/label/1044.png (deflated 76%)
  adding: content/data/label/2369.png (deflated 76%)
  adding: content/data/label/2429.png (deflated 76%)
  adding: content/data/label/756.png (deflated 76%)
  adding: content/data/label/139.png (deflated 76%)
  adding: content/data/label/1529.png (deflated 76%)
  adding: 